In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [7]:
# Create imputer object
imputer = IterativeImputer(max_iter=100, tol=1e-3)

df_train = pd.read_csv('data/iith_foml_2023_train.csv')
for column in df_train.columns:
    if df_train[column].isna().any():
        mean_value = df_train[column].mean()
        df_train[column].fillna(mean_value, inplace=True)
y_train = df_train.iloc[:, -1]
X_train = df_train.iloc[:, :-1]
X_train_imputed = X_train
# Fit the imputer model on the dataset to learn the data patterns
# X_train.drop(columns=['Feature 16', 'Feature 17'], inplace=True)
# imputer.fit(X_train)

# # Transform the dataset to replace missing values
# # Convert back to a DataFrame
# X_train_imputed = pd.DataFrame(imputer.transform(X_train), columns=X_train.columns)

In [8]:
df_test = pd.read_csv('data/iith_foml_2023_test.csv')
for column in df_test.columns:
    if df_test[column].isna().any():
        mean_value = df_test[column].mean()
        df_test[column].fillna(mean_value, inplace=True)
# df_test.drop(columns=['Feature 16', 'Feature 17'], inplace=True)
# X_test = pd.DataFrame(imputer.transform(df_test), columns=df_test.columns)
X_test = df_test

In [9]:
from sklearn.utils.class_weight import compute_sample_weight, compute_class_weight

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test)
# le = LabelEncoder()
# y_train = le.fit_transform(y_train)
# class_wts = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
# class_wts_dict = dict(zip(np.arange(len(class_wts)), class_wts))
# sample_wts = compute_sample_weight(class_weight=class_wts_dict, y=y_train)

In [10]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
X_train, y_train = ros.fit_resample(X_train_scaled, y_train)

from sklearn.ensemble import VotingClassifier, ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
# Initialize models
# extratrees_model = ExtraTreesClassifier(random_state=42)
# randomforest_model = RandomForestClassifier(random_state=42)
# graidentboost_model = GradientBoostingClassifier(random_state=42)

# model = VotingClassifier(
#     estimators=[
#         ('bagging_extratrees', extratrees_model),
#         ('bagging_gradboost', randomforest_model),
#         ('gradientboost_model', graidentboost_model),
#     ], voting='hard'
# )
model = KNeighborsClassifier()
# Train the model on the training data
model.fit(X_train, y_train)

predictions = model.predict(X_test_scaled)

In [11]:
pred_df = pd.DataFrame()
pred_df['Category'] = predictions
pred_df['id'] = pred_df.index + 1
pred_df = pred_df[['id', 'Category']]
pred_df.to_csv('output/voting_oversampled_Knn.csv', index=False)